In [1]:
import openai

In [2]:
from openai import OpenAI

In [4]:
client=OpenAI()

In [5]:
import os

In [1]:
!wget https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py

--2024-08-14 00:04:36--  https://raw.githubusercontent.com/alexeygrigorev/minsearch/main/minsearch.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.108.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3832 (3.7K) [text/plain]
Saving to: ‘minsearch.py’

minsearch.py        100%[===================>]   3.74K  --.-KB/s    in 0s      

2024-08-14 00:04:36 (26.8 MB/s) - ‘minsearch.py’ saved [3832/3832]



In [2]:
import minsearch

In [3]:
import json

In [5]:
with open('documents.json','rt') as f_in:
    docs_raw=json.load(f_in)

In [10]:
documents = []

for course_dict in docs_raw:
    for doc in course_dict['documents']:
        doc['course']=course_dict['course']
        documents.append(doc)

In [11]:
documents[0]

{'text': "The purpose of this document is to capture frequently asked technical questions\nThe exact day and hour of the course will be 15th Jan 2024 at 17h00. The course will start with the first  “Office Hours'' live.1\nSubscribe to course public Google Calendar (it works from Desktop only).\nRegister before the course starts using this link.\nJoin the course Telegram channel with announcements.\nDon’t forget to register in DataTalks.Club's Slack and join the channel.",
 'section': 'General course-related questions',
 'question': 'Course - When will the course start?',
 'course': 'data-engineering-zoomcamp'}

In [13]:
index = minsearch.Index(
    text_fields=["question", "text", "section"],
    keyword_fields=["course"]
)

In [14]:
index.fit(documents)

In [21]:
from openai import OpenAI

In [22]:
client = OpenAI()

In [82]:
def search(query):
    boost = {'question':3.0, 'section':0.5}

    results=index.search(
        query = query,
        filter_dict={'course':'data-engineering-zoomcamp'},
        boost_dict=boost,
        num_results=5)

    return results

In [75]:
def build_prompt(query, search_results):
    prompt_template = """ 
    You're a course teaching assistant. Answer the QUESTION based on the CONTEXT.
    If the CONTEXT doesn't contain the answer, output Y YO QUE SE'
    QUESTION: {question}
    CONTEXT:
    {context}""".strip()
    
    context = ""

    for doc in search_results:
        context=context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer:{doc['text']}\n\n"
    
    prompt=prompt_template.format(question=query, context=context).strip()
    return prompt

In [55]:
def llm(prompt):
    response = client.chat.completions.create(
    model='gpt-4o',
    messages=[{"role": "user", "content": prompt}]
)
    return response.choices[0].message.content

In [85]:
query="can i still enroll the course?"
def rag(query):
    search_results=search(query)
    prompt=build_prompt(query,search_results)
    answer= llm(prompt)
    return answer

In [88]:
rag('when does the course start?')

'The course will start on 15th Jan 2024 at 17h00.'

In [ ]:
ans